### Importing libraries

In [ ]:
import os
import math
import yaml
import time
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import MCFS

from MCFS.utils import path_length, curvature, LineDataUnits, draw_MCPP_solution
from MCFS.isograph import IsoGraph
from MCFS.MMRTC import augmented_isograph, solve_MMRTC_model_GRB, solve_MMRTC_model_SCIP
from MCFS.isolines import LayeredIsoLines
from MCFS.planning import Pathfinder, unified_CFS
from MCFS.solution_refinement import *

from main import MCFS
from baselines import TMC, TMSTCStar

### Defining the MCFS and calc_metrics functions

In [ ]:
def calc_metrics(instance_name, Pi):
    with open(os.path.join("data", "instances", instance_name+".yaml")) as f:
        dict = yaml.load(f, yaml.Loader)
        with open(f"data/polygons/{dict['polygon']}", "rb") as f:
            polygon = pickle.load(f)

    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.axis("equal")

    x_min, y_min, x_max, y_max = polygon.bounds
    x_min, y_min, x_max, y_max = math.floor(x_min), math.floor(y_min), math.ceil(x_max), math.ceil(y_max)
    ax.fill_between(np.arange(x_min, x_max, 0.01), y_min, y_max, color='k')

    ax.fill(polygon.exterior.coords.xy[0], polygon.exterior.coords.xy[1], 'w')
    for interior in polygon.interiors:
        xs, ys = np.array(interior.coords.xy[0]), np.array(interior.coords.xy[1])
        ax.fill(xs, ys, 'k')

    for pi in Pi:
        line = LineDataUnits(pi[:,0], pi[:,1], color=(0,0,1,0.5), linewidth=0.1)
        ax.add_line(line)

    plt.savefig(os.path.join("data", f"{instance_name}.png"), dpi=500)
    plt.close()

    Trans = (1,1,1,0)
    White = (1,1,1,1)
    Black = (0,0,0,1)
    outside_cov = (0.0, 0.0, 0.49803922, 1.0)
    nondup_cov = (0.49803922, 0.49803922, 1.0, 1.0)

    img = plt.imread(os.path.join("data", f"{instance_name}.png"))
    img = img.reshape((-1, 4))
    os.remove(os.path.join("data", f"{instance_name}.png"))

    unique, cnts = np.unique(img, axis=0, return_counts=True)
    dict, area_outside_cov, area_nondup_cov = {}, 0, 0
    for item, cnt in zip(unique, cnts):
        dict[tuple(item)] = cnt
        if (np.around(outside_cov, 2) == np.around(item, 2)).all():
            area_outside_cov += cnt
        if (np.around(nondup_cov, 2) == np.around(item, 2)).all():
            area_nondup_cov += cnt
    
    area_uncovered = dict[White]
    area_polygon = len(img) - dict[Trans] - dict[Black] - area_outside_cov
    area_overlapped = len(img) - dict[Trans] - dict[Black] - area_outside_cov - area_uncovered - area_nondup_cov

    Ls, ks = [], np.array([])
    for i in range(len(Pi)):
        pi = np.array(Pi[i])
        Ls.append(path_length(pi))
        ks = np.vstack(np.sqrt(curvature(pi)))

    makespan, smoothness = max(Ls), np.average(ks)
    COV, OVL = 1-area_uncovered/area_polygon, area_overlapped/area_polygon
    print(f"makespan={makespan:.3f}, smoothness={smoothness:.3f}")
    print(f"COV={COV:.5f}, OVL={OVL:.5f}")

    return makespan, smoothness, COV, OVL


### Running all methonds on all instances

In [ ]:
df = pd.DataFrame()
for c in ["I", "C", "A", "P", "S", "double_torus", "office"]:

    # ---------- TMC ----------
    # t = time.time()
    Pi = TMC.plan(c)
    # print(f"time={time.time()-t:.4f}")
    makespan, smoothness, COV, OVL = calc_metrics(c, Pi)
    df = pd.concat([df, 
        pd.DataFrame([{"method":"TMC", 
                        "makespan":makespan, 
                        "smoothness": smoothness, 
                        "COV": COV, 
                        "OVL": OVL}])], ignore_index=True)

    # ---------- TMSTC* ----------
    # t = time.time()
    Pi = TMSTCStar.plan(c)
    # print(f"time={time.time()-t:.4f}")
    makespan, smoothness, COV, OVL = calc_metrics(c, Pi)
    df = pd.concat([df, 
        pd.DataFrame([{"method":"TMSTC*", 
                        "makespan":makespan, 
                        "smoothness": smoothness, 
                        "COV": COV, 
                        "OVL": OVL}])], ignore_index=True)

    # ---------- MCFS ----------
    for aug, ref in [(False, False), (False, True), (True, False), (True, True)]:
        # t = time.time()
        Pi, metrics = MCFS(c, aug=aug, ref=ref, read_mmrtc_sol=True, write_sol=False)
        # print(f"time={time.time()-t:.4f}")
        makespan, smoothness, COV, OVL = calc_metrics(c, Pi)
        df = pd.concat([df, 
            pd.DataFrame([{"method":f"MCFS{'-Aug' if aug else ''}{'-Ref' if ref else ''}", 
                            "makespan":makespan, 
                            "smoothness": smoothness, 
                            "COV": COV, 
                            "OVL": OVL}])], ignore_index=True)

df.head(6)